In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import tensorflow_probability as tfp
tfd = tfp.distributions

In [13]:
mu1 = np.array([-0.5, 1.2])
sigma1 = np.array([[1.0, 0.5], [0.5, 1.0]])
mu2 = np.array([1.0, -0.1])
sigma2 = np.array([[1.0, 0.2], [0.2, 1.0]])

states = np.array([0, 1, 1, 0, 1])[:, np.newaxis]
obs1 = np.random.multivariate_normal(mu1, sigma1, size=(states.size))
obs2 = np.random.multivariate_normal(mu2, sigma2, size=(states.size))

obs = obs1 * (1-states) + obs2 * states
obs

array([[-0.76245762,  0.85492254],
       [ 1.00578533,  0.61584925],
       [ 0.52215263, -0.12287415],
       [-0.09523309,  1.56628671],
       [-0.30545132, -1.14517517]])

In [14]:
B1 = tfd.MultivariateNormalFullCovariance(loc=mu1, covariance_matrix=sigma1) 
B2 = tfd.MultivariateNormalFullCovariance(loc=mu1, covariance_matrix=sigma2) 

In [15]:
emissions = np.stack([B1.prob(obs), B2.prob(obs)], axis=-1)
emissions

array([[0.17222293, 0.15009381],
       [0.01796157, 0.034757  ],
       [0.01157759, 0.02858735],
       [0.16631633, 0.14344725],
       [0.00337959, 0.00825666]])

In [16]:
emissions.shape

(5, 2)

In [17]:
A = np.array([[0.6, 0.4], [0.1, 0.9]])
P0 = np.array([0.5, 0.5])

In [21]:
# forward variables
alpha = np.zeros((states.size, 2))
alpha[0] = P0 * emissions[0]
for i in range(1, states.size):
    alpha[i] = np.matmul(alpha[i-1], A) * emissions[i]

# backward variables
beta = np.zeros((states.size, 2))
beta[-1] = 1
for i in range(states.size-2, -1, -1):
    beta[i] = np.matmul(A, beta[i+1] * emissions[i+1])

# likelihood
likelihood = np.sum(alpha[-1])

# state posterior probabilities
posterior = alpha * beta / likelihood

print("alpha=\n", alpha)
print("beta=\n", beta)
print("likelihood=\n", likelihood)
print("posterior=\n", posterior)

alpha=
 [[8.61114657e-02 7.50469031e-02]
 [1.06281427e-03 3.54475512e-03]
 [1.14868731e-05 1.03354861e-04]
 [2.86523294e-06 1.40024778e-05]
 [1.05422566e-08 1.13515263e-07]]
beta=
 [[6.13938764e-07 9.48611473e-07]
 [1.92745179e-05 2.92184604e-05]
 [9.77695674e-04 1.09164536e-03]
 [5.33042007e-03 7.76895538e-03]
 [1.00000000e+00 1.00000000e+00]]
likelihood=
 1.2405752005285194e-07
posterior=
 [[0.42615044 0.57384956]
 [0.16512689 0.83487311]
 [0.0905279  0.9094721 ]
 [0.1231114  0.8768886 ]
 [0.08497878 0.91502122]]
